# Data Injection Speed Test
We know that supplying CSV text is the most optimal way to provide data to Vega, but are there significant performance benefits to injecting data into the specification namespace structure before text conversion or after?

In [1]:
∇ Spec←{
  s←0⎕JSON'{}'
  s.opt1←'opt1'
  s.sub←0⎕JSON'{}'
  s.sub.opt1←'sub-opt1'
  s.sub.opt2←'sub-opt2'
  s
}
∇

In [2]:
∇ Inject1←{
  ⍺.data←0⎕JSON'{}'
  ⍺.data.values←⍵ ⎕CSV '' 'S'
  ⍺.data.format←0⎕JSON'{}'
  ⍺.data.format.type←'csv'
  ⍺
}
∇

In [3]:
∇ Inject2←{
  d←'"data": { "values": '
  d,←1⎕JSON ⍵⎕CSV '' 'S'   ⍝ CSV text as JSON string
  d,←', "format": { "type": "csv" }},'
  (1↑⍺),d,(1↓⍺)
}
∇

In [4]:
⎕RL ← 42
data←'this' 'that' 'another'⍪(↓⎕A[?5 4⍴26]),(?5⍴8),⍪?5⍴0
3↑data

┌────┬────┬─────────────┐
│this│that│another │
├────┼────┼─────────────┤
│HEDO│3 │0.00195337832│
├────┼────┼─────────────┤
│XTSQ│2 │0.3582407615 │
└────┴────┴─────────────┘

In [5]:
ViewJSON←{1(⎕JSON⎕OPT'Compact'0)⍵}

In [6]:
s ← Spec 0
st ← ViewJSON s

In [7]:
ViewJSON s Inject1 data

{
 "data": {
 "format": {
 "type": "csv"
 },
 "values": "this,that,another\r\nHEDO,3,0.001953378319754839\r\nXTSQ,2,0.3582407615265167\r\nTWHY,2,0.5741675835424208\r\nXROC,7,0.790488793142451\r\nJHSU,3,0.8759638375585851\r\n"
 },
 "opt1": "opt1",
 "sub": {
 "opt1": "sub-opt1",
 "opt2": "sub-opt2"
 }
}

In [8]:
st Inject2 data

{"data": { "values": "this,that,another\r\nHEDO,3,0.001953378319754839\r\nXTSQ,2,0.3582407615265167\r\nTWHY,2,0.5741675835424208\r\nXROC,7,0.790488793142451\r\nJHSU,3,0.8759638375585851\r\n", "format": { "type": "csv" }},
 "opt1": "opt1",
 "sub": {
 "opt1": "sub-opt1",
 "opt2": "sub-opt2"
 }
}

In [9]:
s ≡⍥(1∘⎕json) 0⎕JSON st Inject2 data

1

In [10]:
]runtime -c "s Inject1 data" "st Inject2 data"

s Inject1 data → 1.1E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* st Inject2 data → 8.5E¯6 | -24% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

Cursory inspection suggests character vector manipulation is faster than namespace conversion. How does larger data table impact?

In [11]:
⎕RL ← 42
n←50000
data←'this' 'that' 'another'⍪(↓⎕A[?n 4⍴26]),(?n⍴8),⍪?n⍴0
3↑data

┌────┬────┬─────────────┐
│this│that│another │
├────┼────┼─────────────┤
│XJYJ│4 │0.00195337832│
├────┼────┼─────────────┤
│EBHB│2 │0.3582407615 │
└────┴────┴─────────────┘

In [12]:
]runtime -c "s Inject1 data" "st Inject2 data"

s Inject1 data → 2.5E¯2 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* st Inject2 data → 3.8E¯2 | +52% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

As data increases, it appears that string concatenation is worse than namespace conversion. This is probably because the assignment makes a reference to the CSV character vector array rather than copying it, whereas takes and drops are heavy copy operations